# Keras  - fit & fit_generator function

The Keras deep learning library includes three separate functions that can be used to train your own models:

    .fit
    .fit_generator
    .train_on_batch

If you’re new to Keras and deep learning you may feel a bit overwhelmed trying to determine which function you’re supposed to use — this confusion is only compounded if you need to work with your own custom data.

*All three of these functions can essentially accomplish the same task — but how they go about doing it is very different.*

Let’s explore 1st 2 functions one-by-one, looking at an example function call, and then discussing how they are different from each other.



## 1) .fit  function -

In [ ]:
model.fit(trainX, trainY, batch_size=32, epochs=50)

Here you can see that we are supplying our training data ( trainX ) and training labels ( trainY ).We then instruct Keras to allow our model to train for 50  epochs with a batch size of 32 .

The call to **.fit**  is making two primary assumptions here:

* Our entire training set can fit into RAM
* There is no data augmentation going on (i.e., there is no need for Keras generators)

Instead, our network will be trained on the raw data. The raw data itself will fit into memory — we have no need to move old batches of data out of RAM and move new batches of data into RAM.

*Furthermore, we will not be manipulating the training data on the fly using data augmentation.*

Note : 
1. For small, simplistic datasets it’s perfectly acceptable to use Keras’ .fit  function.These datasets are often not very challenging and do not require any data augmentation.

2. However, real-world datasets are rarely that simple:

* Real-world datasets are often too large to fit into memory.
* They also tend to be challenging, requiring us to perform data augmentation to avoid overfitting and increase the ability of our model to generalize.

In those situations we need to utilize Keras’ **.fit_generator**  function:

## 2) .fit_generator function -

In [ ]:

# initialize the number of epochs and batch size
EPOCHS = 100
BS = 32
 
# construct the training image generator for data augmentation
aug = ImageDataGenerator(rotation_range=20, zoom_range=0.15, width_shift_range=0.2, height_shift_range=0.2,
                         shear_range=0.15, horizontal_flip=True, fill_mode="nearest")
 
# train the network
H = model.fit_generator(aug.flow(trainX, trainY, batch_size=BS),
validation_data=(testX, testY), steps_per_epoch=len(trainX) // BS,epochs=EPOCHS)

# // divides with integral result (ignore the remainder)

Here we start by first initializing the number of epochs we are going to train our network for along with the batch size. We then initialize *aug* , a Keras **ImageDataGenerator**  object that is used to apply data augmentation, randomly translating, rotating, resizing, etc. images on the fly.

* Performing data augmentation is a form of regularization, enabling our model to generalize better.

* However, applying data augmentation implies that our training data is no longer “static” — the data is constantly changing.

* Each new batch of data is randomly adjusted according to the parameters supplied to ImageDataGenerator .

Thus, we now need to utilize Keras’ **.fit_generator**  function to train our model.

As the name suggests, the .fit_generator  function assumes there is an underlying function (a Python generator) that is generating the data for it.

Internally, Keras is using the following process when training a model with .fit_generator :

1. Keras calls the generator function supplied to .fit_generator  (in this case, *aug.flow* ).
2. The generator function yields a batch of size BS  to the .fit_generator  function.
3. The .fit_generator  function accepts the batch of data, performs backpropagation, and updates the weights in our model.
4. This process is repeated until we have reached the desired number of epochs.

You’ll notice we now need to supply a *steps_per_epoch*  parameter when calling **.fit_generator**  (the .fit  method had no such parameter).

Why do we need steps_per_epoch ?

* Keep in mind that a Keras data generator is meant to loop infinitely — it should never return or exit.

Since the function is intended to loop infinitely, Keras has no ability to determine when one epoch starts and a new epoch begins.

Therefore, we compute the steps_per_epoch  value as the total number of training data points divided by the batch size. Once Keras hits this step count it knows that it’s a new epoch and stops calling the image generator

## Summary -

In this tutorial you learned the differences between Keras’ 2 primary functions used to train a deep neural network:

* **.fit** : Used when the entire training dataset can fit into memory and no data augmentation is applied.

* **.fit_generator** : Should be used when either - *when the dataset is too large to fit into memory* or *data augmentation needs to be applied,* or *in any situation when it’s more convenient to yield training data in batches (i.e., using the flow_from_directory  function).* 
